## Reproducing Results 

Most of the experiments involved the DomainNet and Visda2017 datasets, which require us to fine-tune Resnet50-101 architectures. Each run takes a while (up to 8 hours) and requires a GPU accelerator. Therefore in this notebook I demonstrate how to rerun the experiments with different hyperparameters on a slurm cluster, like Izar on Scitas. 

The launch.py script contains a class called experimentBuilder, with methods for setting hyperparameters, such as setAlpha. When initializing an experiment builder, you pass it a list of paths to .yaml files that contain all the necessary configuration (see experiment_registry) for the experiments you wish to run. These base experiment configuration files contain details such as the architecture, source and target domain datasets, and so on. Calling a builder method like "setAlpha" will produce versions of all the base_experiments with all the variations of the parameters you pass the builder method. 

For example, calling .setAlpha([1,2,3]).setEta([2,3]) will make a total of 6 experiments with all the combinations of the Alpha and Eta parameters. 

After calling these builder methods, you can launch all experiments by calling .launch() on the experiment builder class. When this is done, it will create a dict of the config and pass it to a batch submission script for each of the generated experiments, which in turn passes it as an input to the run_experiment.py script, which will execute the experiment for that configuratio on one of the cluster nodes. 

All the metrics are logged on weights and biases, and can be accessed with the weights and biases API for plotting and analysis. Below I give a brief demo of how the slurm submission script works.

In [ ]:
#let's first change the working directory to /robust-domain-adaptation
#if you're not running this notebook from there, run the command below.
%cd .. 

In [ ]:
import os
base_path = "/home/mielonen/robust-domain-adaptation"

#select the experiments on which all the runs are based, can be any file on the experiment registry directory.
base_experiments = [
    "experiment_registry/mcc-domain.yaml"
]

#append the base path
base_experiments = [os.path.join(base_path, e) for e in base_experiments]

In [ ]:
import yaml
def yamlToDict(full_path):
    with open(full_path, "r") as stream:
        dict_ = yaml.safe_load(stream)
    return dict_

In [ ]:
#here's a copy of the experimentBuilder class from launch.py. 
#the idea is to create a set of nested generators using the builder design pattern. 
# .launch() will then create an os command for each of the dictionaries it receives from the final generator. 

class experimentBuilder:

    """
    Usage: 
    experimentBuilder(base_experiments).setSeed([1,2,3]).setCfol(True).launch() 

    Explanation:
    For each base experiment set seeds to [1, 2, 3], Cfol to true, and submit everything as a slurm batch job. 
    """

    def __init__(self, fnames):

        self.base_list = [yamlToDict(name) for name in fnames]

        def base():
            for dic in self.base_list: 
                yield dic

        self.generator = base()

    def setSeed(self, list_):

        def new(base):
            for dict_ in base:
                for seed in list_:
                    newDict = dict_
                    newDict["experiment"]["global_params"]["seed"] = seed
                    yield newDict

        self.generator = new(self.generator)
        return self


    def setGamma(self, list_):

        def new(base):
            for dict_ in base:
                for gamma in list_:
                    newDict = dict_
                    assert(gamma > 0)
                    assert(gamma < 1)
                    newDict["experiment"]["global_params"]["cfol_gamma"] = gamma
                    yield newDict

        self.generator = new(self.generator)
        return self

    def setMethod(self, val):
    
        def new(base):
            for dict_ in base:
                newDict = dict_
                newDict["experiment"]["global_params"]["reweight_method"] = val
                yield newDict

        self.generator = new(self.generator)
        return self

    def setEta(self, list_):

        def new(base):
            for dict_ in base:
                for eta in list_:
                    newDict = dict_
                    newDict["experiment"]["global_params"]["cfol_eta"] = eta
                    yield newDict

        self.generator = new(self.generator)
        return self

    def setAlpha(self, list_):

        def new(base):
            for dict_ in base:
                for alpha in list_:
                    newDict = dict_
                    newDict["experiment"]["global_params"]["cvar_alpha"] = alpha
                    yield newDict

        self.generator = new(self.generator)
        return self

    def launch(self):
        a = self.generator
        for i in a:
            jsonString = json.dumps(i)
            str_cmd = f"sbatch submit_job.run '{jsonString}'"
            os.system(str_cmd)

In [ ]:
#this sets the seed to 12, the reweighting method to lcvar, and lcvar alpha to 0.80. "launch" calls sbatch and submits a job with a json version of the generated config dictionary.
experimentBuilder(base_experiments).setSeed([12]).setMethod("lcvar").setAlpha([0.80]).launch()

In [ ]:
#if you want to go crazy with hyperparameter tuning, you can also make the following call: 
experimentBuilder(base_experiments).setSeed([12,13,14]).setMethod("lcvar").setAlpha([0.5,0.7,0.9]).launch()
experimentBuilder(base_experiments).setSeed([12,13,14]).setMethod("cfol").setEta([0.01,0.001,0.000001]).launch()
#if base_experiments were to contain all 6 base versions of the experiments (2 different datasets for each of the 3 methods), 
#these two lines would submit a total of 108 slurm jobs, for each of the possible variations of parameters. 

In order to reproduce all results, you would set base_experiments to a list of all files in the experiment_registry. Then, you would make one call for setMethod = lcvar, one for setMethod = cfol, giving a total of 12 experiments. To gather statistics you would select at least 3 different seeds, for a total of 36, and so on. If you have any questions, feel free to reach out to me at: 

eelis.mielonen@epfl.ch